In [31]:
#Importing the necessary libraries

from pyspark.sql import SparkSession
from pyspark.sql.functions import  rank, when, row_number, col, desc, year, sum
from pyspark.sql.window import Window

Initiating the spark session

In [32]:
spark = SparkSession.builder.appName('Individual_Spark')\
        .config('spark.driver.extraClassPath','/Users/anujkhadka/Fusemachines47/AllSPARK/Apache-Spark/spark-3.4.1-bin-hadoop3/jars/postgresql-42.6.0.jar')\
        .getOrCreate()

Loading the datasets from Postgres

In [33]:
revised_final = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/postgres") \
    .option("dbtable", "revised_final") \
    .option("user", "ujjwal") \
    .option("password", "ujjwal12345") \
    .option("driver", "org.postgresql.Driver") \
    .load()


In [34]:
goods_final = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/postgres") \
    .option("dbtable", "goods_final") \
    .option("user", "ujjwal") \
    .option("password", "ujjwal12345") \
    .option("driver", "org.postgresql.Driver") \
    .load()


In [35]:
country_final = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/postgres") \
    .option("dbtable", "country_final") \
    .option("user", "ujjwal") \
    .option("password", "ujjwal12345") \
    .option("driver", "org.postgresql.Driver") \
    .load()


In [36]:
services_final= spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/postgres") \
    .option("dbtable", "services_final") \
    .option("user", "ujjwal") \
    .option("password", "ujjwal12345") \
    .option("driver", "org.postgresql.Driver") \
    .load()

Q.1 Compute the following metrics for each country:

The sum of total export values.

The sum of total import values.

The difference between the sum of export and import values as ‘Trade balance’.

Now, Join  'revised_final" table with the 'country_final' table  & filter the result to select the peak countries with the highest trade balance.

In [37]:
# Extracting / filtering year and Accounts 
filtered_df = revised_final.filter((year(col("time_ref")) == 2023) & (col("account").isin("Exports", "Imports")))

trade_balance_df = filtered_df.groupBy("country_code").agg(
    sum(
        when(col("account") == "Exports", col("value"))
        .otherwise(-col("value"))
    ).alias("trade_balance"),
    sum(
        when(col("account") == "Exports", col("value"))
        .otherwise(0)
    ).alias("export_value"),
    sum(
        when(col("account") == "Imports", col("value"))
        .otherwise(0)
    ).alias("import_value")
)
# Introducing window specifications for ranking countries (desc)
window_spec = Window.orderBy(desc("trade_balance"))

#using the rank (W.F) to calculate the row number over window specification
ranked_df = trade_balance_df.withColumn("rank", row_number().over(window_spec))

top_countries_df = ranked_df.filter(col("rank") <= 20)

result_df = top_countries_df.join(
    country_final,
    top_countries_df["country_code"] == country_final["country_code"],
    "inner"
).select(
    "country_label", 
    "rank",
    "export_value",
    "import_value",
    "trade_balance" 
)

result_df.show(truncate = False)


23/09/13 14:28:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/13 14:28:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/13 14:28:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/13 14:28:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/13 14:28:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/13 14:28:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------------------------------------+----+---------------+---------------+-------------+
|country_label                            |rank|export_value   |import_value   |trade_balance|
+-----------------------------------------+----+---------------+---------------+-------------+
|China, People's Republic of              |1   |3.3683951304E10|2.378890331E10 |9.895047994E9|
|United States of America                 |2   |2.049617077E10 |1.8345995471E10|2.150175299E9|
|Algeria                                  |3   |1.893492197E9  |1332102.0      |1.892160095E9|
|Hong Kong (Special Administrative Region)|5   |1.963651478E9  |4.86231181E8   |1.477420297E9|
|United Arab Emirates                     |6   |1.598095552E9  |2.36545099E8   |1.361550453E9|
|Taiwan                                   |7   |2.907311273E9  |1.645299258E9  |1.262012015E9|
|Saudi Arabia                             |8   |1.526632326E9  |2.91180509E8   |1.235451817E9|
|Destination Unknown - EU                 |9   |9.

23/09/13 14:28:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/13 14:28:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/13 14:28:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/13 14:28:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Dumping the output back to Postgres

In [38]:
result_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'q1_output1', user = 'ujjwal',password='ujjwal12345').mode('overwrite').save()

23/09/13 14:28:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/13 14:28:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/13 14:28:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/13 14:28:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/13 14:28:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/13 14:28:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/13 1

Q.2 Show the comparative study of the average transactions of 'account' column (export & import) for different product types (Goods & Services) from 2020 to 2023 combined in a table and show it in columns using Pivot DataFrame Transformations.

In [39]:
filtered_df = revised_final.filter(
    (year(col("time_ref")) >= 2020) & (col("account").isin("Exports")) & (col("product_type") == "Goods")
)

#Extracting year & adding to filtered_df in a new column "year" using 'year'function
filtered_df = filtered_df.withColumn("year", year("time_ref"))

#Grouping filtered_df by country_code into total_account_df & then pivoting it to transform years into separate columns.
total_account_df = filtered_df.groupBy("country_code").pivot("year").agg({"value": "avg"})

#Joining total_account_df with country_final using 'country_code' key
result_df1 = total_account_df.join(
    country_final,
    total_account_df["country_code"] == country_final["country_code"],
    "left"
).select(
    "country_label",
    "2020",
    "2021",
    "2022",
    "2023"
)

result_df1.show(truncate = False)

+------------------------+--------------------+--------------------+--------------------+------------------+
|country_label           |2020                |2021                |2022                |2023              |
+------------------------+--------------------+--------------------+--------------------+------------------+
|Algeria                 |2.0381960916666668E7|1.4288990327586208E7|6.2716360384615384E7|1.050414865E8     |
|Myanmar                 |1165648.2452830188  |2292425.3766233767  |1431446.8101265824  |423120.75         |
|Lithuania               |141195.30841121497  |215586.06           |280860.5533980582   |248225.86363636365|
|Turks and Caicos Islands|5298.5              |50809.0             |108757.33333333333  |41095.5           |
|Cote d'Ivoire           |1198616.0625        |1233945.5294117648  |1690797.0810810812  |1675611.5         |
|Finland                 |177018.1894736842   |105155.19656019656  |95115.64864864865   |171087.97014925373|
|Azerbaijan        

Dumping the output back to Postgres

In [40]:
# output for first question write
result_df1.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'q2_export_goods', user = 'ujjwal',password='ujjwal12345').mode('overwrite').save()

In [41]:
filtered_df = revised_final.filter(
    (year(col("time_ref")) >= 2020) & (col("account").isin("Exports")) & (col("product_type") == "Services")
)

filtered_df = filtered_df.withColumn("year", year("time_ref"))

total_account_df = filtered_df.groupBy("country_code").pivot("year").agg({"value": "avg"})

#Joining total_account_df with country_final using 'country_code' key
result_df2 = total_account_df.join(
    country_final,
    total_account_df["country_code"] == country_final["country_code"],
    "left"
).select(
    "country_label",
    "2020",
    "2021",
    "2022",
    "2023"
)

result_df2.show(truncate = False)


+------------------------+------------------+------------------+------------------+------------------+
|country_label           |2020              |2021              |2022              |2023              |
+------------------------+------------------+------------------+------------------+------------------+
|Algeria                 |91256.0625        |113624.70588235294|97372.06896551725 |171590.0          |
|Myanmar                 |1072804.8         |684503.4545454546 |387020.3846153846 |1425783.1875      |
|Lithuania               |174964.0          |94918.84          |184666.44444444444|383944.25         |
|Turks and Caicos Islands|50448.0           |17230.0           |26239.21052631579 |75184.33333333333 |
|Cote d'Ivoire           |73614.2           |16262.5           |25023.333333333332|49710.9           |
|Finland                 |322475.4666666667 |174508.2068965517 |772234.6333333333 |2018030.125       |
|Azerbaijan              |56326.692307692305|10418.07142857143 |14341.7  

Dumping the output back to Postgres

In [42]:
result_df1.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'q2_export_services', user = 'ujjwal',password='ujjwal12345').mode('overwrite').save()

In [43]:
filtered_df = revised_final.filter(
    (year(col("time_ref")) >= 2020) & (col("account").isin("Imports")) & (col("product_type") == "Goods")
)

#Extracting year & adding to filtered_df in a new column "year" using 'year'function
filtered_df = filtered_df.withColumn("year", year("time_ref"))

#Grouping filtered_df by country_code into total_account_df & then pivoting it to transform years into separate columns.
total_account_df = filtered_df.groupBy("country_code").pivot("year").agg({"value": "avg"})

#Joining total_account_df with country_final using 'country_code' key
result_df2 = total_account_df.join(
    country_final,
    total_account_df["country_code"] == country_final["country_code"],
    "left"
).select(
    "country_label",
    "2020",
    "2021",
    "2022",
    "2023"
)

result_df2.show(truncate = False)

+------------------------+------------------+------------------+------------------+------------------+
|country_label           |2020              |2021              |2022              |2023              |
+------------------------+------------------+------------------+------------------+------------------+
|Algeria                 |13510.714285714286|2768498.234042553 |750324.0882352941 |21886.5           |
|Myanmar                 |72032.75480769231 |77712.00735294117 |85455.783847981   |67372.61209964413 |
|Lithuania               |65892.56544502618 |116902.9896373057 |124374.63339222615|85715.64669421487 |
|Turks and Caicos Islands|385.2             |932.0             |294.0             |83.0              |
|Cote d'Ivoire           |541077.0          |124058.88461538461|92846.18181818182 |52495.166666666664|
|Finland                 |333713.72647058824|409803.3964365256 |468028.9968895801 |538715.0324074074 |
|Azerbaijan              |1520.25           |56132.166666666664|29178.125

Dumping the output back to Postgres

In [44]:
result_df1.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'q2_import_goods', user = 'ujjwal',password='ujjwal12345').mode('overwrite').save()

In [45]:
filtered_df = revised_final.filter(
    (year(col("time_ref")) >= 2020) & (col("account").isin("Imports")) & (col("product_type") == "Services")
)

filtered_df = filtered_df.withColumn("year", year("time_ref"))

total_account_df = filtered_df.groupBy("country_code").pivot("year").agg({"value": "avg"})

#Joining total_account_df with country_final using 'country_code' key
result_df3 = total_account_df.join(
    country_final,
    total_account_df["country_code"] == country_final["country_code"],
    "left"
).select(
    "country_label",
    "2020",
    "2021",
    "2022",
    "2023"
)

result_df3.show(truncate = False)

+------------------------+------------------+------------------+------------------+------------------+
|country_label           |2020              |2021              |2022              |2023              |
+------------------------+------------------+------------------+------------------+------------------+
|Algeria                 |51978.0           |81631.4           |64236.35          |115701.0          |
|Myanmar                 |178865.5          |85440.38461538461 |83474.4705882353  |373504.625        |
|Lithuania               |320175.0          |358652.2          |105708.11111111111|190071.4          |
|Turks and Caicos Islands|56250.28571428572 |21635.85714285714 |29759.15          |93889.63636363637 |
|Cote d'Ivoire           |83574.33333333333 |120156.0          |22658.69230769231 |35077.0           |
|Finland                 |470337.5          |315817.3888888889 |361824.2380952381 |378502.2727272727 |
|Azerbaijan              |135033.7142857143 |153949.66666666666|45845.631

Dumping the output back to Postgres

In [46]:
result_df3.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'q2_import_services', user = 'ujjwal',password='ujjwal12345').mode('overwrite').save()

Converting csv file to Parquet

In [47]:
parquetFilePath = "./ParquetFile/"

revised_final.write.parquet(f"{parquetFilePath}/revised_final", mode="overwrite")